# Global Minimial Viable Products (1)
This is only for CONUS currently as the potential runoff product is to large to be an asset. 

Combine:
* SMAP available porosity
* ERA5-Merit potential runoff
* MERIT terrain slope
* MERIT upstream contributing area (needs to be less than some threshold)
* (ERA5 Moisture values)
* USDA soil texture class

In to various "products".

In [1]:
import ee
import numpy as np

import sys
sys.path.insert(1, '../datasets/SMAP-USDA/')  # until we package this
from earthshot import water_viz as vis
from earthshot import water_common as common
from earthshot import normalize as norm
from statistics import mean
import folium
from folium import plugins

Loading BokehJS ...

In [2]:
#ee.Authenticate()
ee.Initialize()

In [3]:
# Some common parameters
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
days_per_month = [
    31, 28, 31, 30, 31, 30,
    31, 31, 30, 31, 30, 31] 
seconds_per_month = [ dd* 24*60*60 for dd in days_per_month ]
month = 4
month_name = month_names[month - 1]

bbox = common.bboxes()['conus']

# SMAP-USDA Available Porosity
More available porosity is better.

In [10]:
# Get a single example image, available porosity in mm
smap_usda_clim = ee.ImageCollection('users/jamesmcc/smap_usda_climatology')
avail_porosity = smap_usda_clim.filter(ee.Filter.eq('band', 'avail_porosity_mm'))
avail_porosity_img = avail_porosity.sum()

avail_porosity_img_scaled = (norm.img_scale(
    avail_porosity_img, area_of_interest=common.bboxes()['conus']))

In [5]:
# smap_range = norm.img_range(img, area_of_interest=common.bboxes()['conus'])
# smap_range

## ERA5 Runoff

In [11]:
# Get a single example image 
runoff_clim = ee.ImageCollection(
    'users/jamesmcc/era5_sfc_runoff_conus_climatology')

runoff_img = runoff_clim.sum()

runoff_img_scaled = norm.img_scale(
    runoff_img, 
    area_of_interest=common.bboxes()['conus'])

runoff_range = norm.img_range(runoff_img_scaled, area_of_interest = bbox)
print(runoff_range)


[0, 1]


## MERIT Terrain Slope

In [17]:
#include slope as a factor in scoring product 1
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
slope_mask = slope_img.lte(10).And(slope_img.gt(0))
slope_img_masked = slope_img.updateMask(slope_mask).pow(-1)
slope_img_scaled = norm.img_scale(slope_img_masked, area_of_interest=common.bboxes()['conus'])

# Soil Types

In [13]:
#incorporate soil types as a factor in scoring land parcels
soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

In [14]:
#categorizing soil types and depths based on retaining plant-available water, grouping top soils and bottom soils together
top_soils = [5,7,8,10]
medium_soils = [2,4,6,9]
low_soils = [1,3,11,12]
soil_0 = soil_types.expression(
        "(b('b0') == 12) ? 1.0" +
        ": (b('b0') == 11) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 9) ? 1.0" +
        ": (b('b0') == 8) ? 0.7" +
        ": (b('b0') == 7) ? 0.7" +
        ": (b('b0') == 6) ? 0.7" +
        ": (b('b0') == 5) ? 0.7" +
        ": (b('b0') == 4) ? 0.4" +
        ": (b('b0') == 3) ? 0.4" +
        ": (b('b0') == 2) ? 0.4" +
        ": (b('b0') == 1) ? 0.4" +
        ": 0")

soil_10 = soil_types.expression(
        "(b('b10') == 12) ? 1.0" +
        ": (b('b10') == 11) ? 1.0" +
        ": (b('b10') == 10) ? 1.0" +
        ": (b('b10') == 9) ? 1.0" +
        ": (b('b10') == 8) ? 0.7" +
        ": (b('b10') == 7) ? 0.7" +
        ": (b('b10') == 6) ? 0.7" +
        ": (b('b10') == 5) ? 0.7" +
        ": (b('b10') == 4) ? 0.4" +
        ": (b('b10') == 3) ? 0.4" +
        ": (b('b10') == 2) ? 0.4" +
        ": (b('b10') == 1) ? 0.4" +
        ": 0")

soil_30 = soil_types.expression(
        "(b('b30') == 12) ? 1.0" +
        ": (b('b30') == 11) ? 1.0" +
        ": (b('b30') == 10) ? 1.0" +
        ": (b('b30') == 9) ? 1.0" +
        ": (b('b30') == 8) ? 0.7" +
        ": (b('b30') == 7) ? 0.7" +
        ": (b('b30') == 6) ? 0.7" +
        ": (b('b30') == 5) ? 0.7" +
        ": (b('b30') == 4) ? 0.4" +
        ": (b('b30') == 3) ? 0.4" +
        ": (b('b30') == 2) ? 0.4" +
        ": (b('b30') == 1) ? 0.4" +
        ": 0")

soil_60 = soil_types.expression(
        "(b('b60') == 5) ? 1.0" +
        ": (b('b60') == 7) ? 1.0" +
        ": (b('b60') == 8) ? 1.0" +
        ": (b('b60') == 10) ? 1.0" +
        ": (b('b60') == 2) ? 0.7" +
        ": (b('b60') == 4) ? 0.7" +
        ": (b('b60') == 6) ? 0.7" +
        ": (b('b60') == 9) ? 0.7" +
        ": (b('b60') == 1) ? 0.3" +
        ": (b('b60') == 3) ? 0.3" +
        ": (b('b60') == 11) ? 0.3" +
        ": (b('b60') == 12) ? 0.3" +
        ": 0")

soil_100 = soil_types.expression(
        "(b('b100') == 5) ? 1.0" +
        ": (b('b100') == 7) ? 1.0" +
        ": (b('b100') == 8) ? 1.0" +
        ": (b('b100') == 10) ? 1.0" +
        ": (b('b100') == 2) ? 0.7" +
        ": (b('b100') == 4) ? 0.7" +
        ": (b('b100') == 6) ? 0.7" +
        ": (b('b100') == 9) ? 0.7" +
        ": (b('b100') == 1) ? 0.3" +
        ": (b('b100') == 3) ? 0.3" +
        ": (b('b100') == 11) ? 0.3" +
        ": (b('b100') == 12) ? 0.3" +
        ": 0")

soil_200 = soil_types.expression(
        "(b('b200') == 5) ? 1.0" +
        ": (b('b200') == 7) ? 1.0" +
        ": (b('b200') == 8) ? 1.0" +
        ": (b('b200') == 10) ? 1.0" +
        ": (b('b200') == 2) ? 0.7" +
        ": (b('b200') == 4) ? 0.7" +
        ": (b('b200') == 6) ? 0.7" +
        ": (b('b200') == 9) ? 0.7" +
        ": (b('b200') == 1) ? 0.3" +
        ": (b('b200') == 3) ? 0.3" +
        ": (b('b200') == 11) ? 0.3" +
        ": (b('b200') == 12) ? 0.3" +
        ": 0")

top_soils = soil_0.expression('top_soil + soil_10 + soil_30',
                             {'top_soil': soil_0.select('constant'),
                             'soil_10': soil_10.select('constant'),
                             'soil_30': soil_30.select('constant')})

bottom_soils = soil_60.expression('soil_60 + soil_100 + soil_200',
                                 {'soil_60': soil_60.select('constant'),
                                  'soil_100': soil_100.select('constant'),
                                  'soil_200': soil_200.select('constant')})

In [15]:
#scaling top soils and bottom soils
top_soils_scaled = norm.img_scale(top_soils, area_of_interest=common.bboxes()['conus'])
bottom_soils_scaled = norm.img_scale(bottom_soils, area_of_interest=common.bboxes()['conus'])

## Product 1
Simply give higher scores to places that have both high potential runoff and available porosity (and with low moisture), in only areas with acceptable slopes

In [20]:
score = (runoff_img_scaled.multiply(.25)
         .add(avail_porosity_img_scaled).multiply(.25)
         .add(top_soils_scaled.multiply(.125))
         .add(bottom_soils_scaled.multiply(.125))
         .add(slope_img_scaled.multiply(.25)))
score = norm.img_scale(score, area_of_interest=bbox).rename('score')
score_range = norm.img_range(score, area_of_interest=bbox)

In [21]:
#understanding product_1 range


palette_name = 'RdYlBu'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum= score_range[0], maximum= score_range[1])

box_corners = bbox.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

vis_params = {
    'min': score_range[0], 'max': score_range[1], 'dimensions': 512,
    'palette': palette}



In [29]:
#creating the map, visualizing parcel score data for parcels in douglas and coos counties, option to visualize soil texture as well

the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=5, height=1500)

the_map.add_ee_layer(score, vis_params, name = 'PRZ Score')

vis.folium_display(the_map)



In [67]:
collection_name_d = (
    'users/amgadellaboudy/product_1_douglas')
product_1_d_asset = ee.data.createAsset(
    {'type': 'ImageCollection'}, collection_name_d)
description_d= 'Douglas_County_Water_Parcel_Score'

collection_name_c = (
    'users/amgadellaboudy/product_1_coos')
product_1_c_asset = ee.data.createAsset(
    {'type': 'ImageCollection'}, collection_name_c)
description_c= 'Coos_County_Water_Parcel_Score'
oregon_box = [[-124.1, 43.38], [-124.1, 42.88], [-123.6, 42.88], [-123.6, 43.38]]

xx_d = ee.batch.Export.image.toAsset(parcel_score_douglas_2, description = description_d, assetId = product_1_d_asset['id'] + '/' + description_d, region=oregon_box, scale = 90 )
xx_c = ee.batch.Export.image.toAsset(parcel_score_coos_2, description = description_c, assetId = product_1_c_asset['id'] + '/' + description_c, region= oregon_box, scale = 90)
xx_d.start()
xx_c.start()

## Product 2
Ratio of potential_runoff:available_porosity?

In [13]:
runoff_pixel_area_m2 = pot_runoff_img.pixelArea()
pot_runoff_img_mm = pot_runoff_img.divide(runoff_pixel_area_m2).multiply(seconds_per_month[month-1] * 1000)
# pot_runoff_mask = pot_runoff_img_mm.gte(1000)
avail_porosity_mask = avail_porosity_img.gte(50)

In [14]:
norm.img_range(pot_runoff_img_mm.updateMask(runoff_mask), area_of_interest=bbox)
#norm.img_range(pot_runoff_img_mm, area_of_interest=bbox)

[16.847965699571173, 1162.8066667861892]

In [15]:
norm.img_range(runoff_pixel_area_m2, area_of_interest=bbox)

[1147915.625, 1592437.625]

In [16]:
norm.img_range(avail_porosity_img, area_of_interest=bbox)

[0, 205.34410095214844]

In [17]:
product_2_raw = (
    pot_runoff_img_mm
    .divide(avail_porosity_img)
    .updateMask(slope_mask)
    .updateMask(avail_porosity_mask)
    .updateMask(runoff_mask))
product_2_mask = product_2_raw.gte(1)
product_2 = product_2_raw.updateMask(product_2_mask)

In [18]:
product_2_range = norm.img_range(product_2, area_of_interest=bbox)


palette_name = 'BrBG'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum=product_2_range[0], maximum=product_2_range[1])

box_corners = bbox.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

vis_params = {
    'min': product_2_range[0], 'max': product_2_range[1], 'dimensions': 512,
    'palette': palette}

In [19]:
the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=4, height=500)
the_map.add_ee_layer(product_2, vis_params, 'Product 2: ' + month_name)
vis.folium_display(the_map)